# Задание 1
Пусть X ∼ F(x) — случайная величина с произвольным распределением, неизвестным математическим ожиданием 𝔼X и известной дисперсией 𝔻X. Как выглядит доверительный интервал для 𝔼X с приближённым уровнем доверия 100(1−α)%?

> $ 𝔼X \pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{𝔻X}{n}}$

# Задание 2

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Будьте осторожны при использовании метода `std()`! Дело в том, что у объекта numpy он по умолчанию вычисляется как $\sqrt{\frac1{n}\sum_{i=1}^n\left(X_i-\bar{X}\right)^2}$
а у объекта pandas — как $\sqrt{\frac1{n-1}\sum_{i=1}^n\left(X_i-\bar{X}\right)^2}$.

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае `std()`, можно всегда использовать `std(ddof=1)` (ddof — difference in degrees of freedom), тогда нормировка всегда будет на `n-1`.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('water.txt', delimiter='\t')
data.head()

location        town  mortality  hardness
0    South        Bath       1247       105
1    North  Birkenhead       1668        17
2    South  Birmingham       1466         5
3    North   Blackburn       1800        14
4    North   Blackpool       1609        18

In [3]:
from statsmodels.stats.weightstats import _tconfint_generic

def confidence_interval(data):
    mean = data.mean()
    mean_std = data.std(ddof=1) /  np.sqrt(data.shape[0])

    return _tconfint_generic(mean, mean_std, data.shape[0]-1 , 0.05, 'two-sided')

In [5]:
confidence_interval(data.mortality)

(1476.0833413552848, 1572.2117406119285)

In [15]:
print("Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки. Ответ -", round(confidence_interval(data.mortality)[0], 4))

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки. Ответ - 1476.0833


# Задание 3

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [17]:
confidence_interval(data[data.location == 'South'].mortality)

(1320.1517462936238, 1433.463638321761)

In [23]:
print("На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки. Ответ -", round(confidence_interval(data[data.location == 'South'].mortality)[1], 4))

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки. Ответ - 1433.4636


# Задание 4

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

> Интервалы не пересекаются; видимо, средняя смертность на севере и на юге существенно разная

In [19]:
confidence_interval(data[data.location == 'North'].mortality)

(1586.5605251961385, 1680.6394748038613)

# Задание 5

Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

> Не пересекаются

In [20]:
confidence_interval(data[data.location == 'South'].hardness)

(53.467198692036106, 86.07126284642544)

In [21]:
confidence_interval(data[data.location == 'North'].hardness)

(21.42248728572426, 39.37751271427574)

# Задание 6

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^{2}$:

$P(\bar{X}_n - z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}} \leqslant \mu \leqslant \bar{X}_n + z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}) = 1 - \alpha$

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью ±0.1?

**Решение**

$z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}} = 0.1$

$\frac{z_{1-\frac{\alpha}{2}}\sigma}{0.1} = \sqrt{n}$

$ n = (\frac{z_{1-\frac{\alpha}{2}}\sigma}{0.1})^{2} = 100z_{1-\frac{\alpha}{2}}^{2} \approx 385$

Где $\alpha = 0.05$ и $\sigma=1$

> 385

In [24]:
from scipy.stats import norm
z = norm.ppf(1-0.05/2)
np.ceil(100 * pow(z, 2))

385.0

# Задание 7

Объём выборки, необходимый для построения доверительного интервала заданной ширины:

 
> [] увеличивается с увеличением требуемой ширины

> [x] увеличивается с уменьшением требуемой ширины

> [] увеличивается с ростом α

> [x] уменьшается с ростом α

> [x] увеличивается с ростом дисперсии выборки

> [] уменьшается с ростом дисперсии выборки